In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-airline-sentiment/Tweets.csv
/kaggle/input/twitter-airline-sentiment/database.sqlite


In [2]:
path="/kaggle/input/twitter-airline-sentiment/database.sqlite"
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sqlite3
conn=sqlite3.connect(path)

In [4]:
import pandas as pd
data=pd.read_sql_query("SELECT * FROM tweets",conn)

In [5]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567588278875213824,neutral,1.0,,,Delta,,JetBlueNews,,0,@JetBlue's new CEO seeks the right balance to ...,,2015-02-16 23:36:05 -0800,USA,Sydney
1,567590027375702016,negative,1.0,Can't Tell,0.6503,Delta,,nesi_1992,,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,,2015-02-16 23:43:02 -0800,undecided,Pacific Time (US & Canada)
2,567591480085463040,negative,1.0,Late Flight,0.346,United,,CPoutloud,,0,@united yes. We waited in line for almost an h...,,2015-02-16 23:48:48 -0800,"Washington, DC",
3,567592368451248130,negative,1.0,Late Flight,1,United,,brenduch,,0,@united the we got into the gate at IAH on tim...,,2015-02-16 23:52:20 -0800,,Buenos Aires
4,567594449874587648,negative,1.0,Customer Service Issue,0.3451,Southwest,,VahidESQ,,0,@SouthwestAir its cool that my bags take a bit...,,2015-02-17 00:00:36 -0800,"Los Angeles, CA",Pacific Time (US & Canada)


In [6]:
data.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [7]:
df=data[["text","airline_sentiment"]]

In [8]:
df

,text,airline_sentiment
0,@JetBlue's new CEO seeks the right balance to ...,neutral
1,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,negative
2,@united yes. We waited in line for almost an h...,negative
3,@united the we got into the gate at IAH on tim...,negative
4,@SouthwestAir its cool that my bags take a bit...,negative
...,...,...
14480,@JetBlue I hope so because I fly very often an...,neutral
14481,@USAirways is a DM possible if you aren't foll...,neutral
14482,@JetBlue Yesterday on my way from EWR to FLL j...,positive
14483,@USAirways and when will one of these agents b...,negative


# Preprocessing the data : 

In [ ]:
import re
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



In [ ]:
def preprocess_text(text):
  #making the text to lowercase
  text=text.lower()
  #removing the url from the data in columns
  text = re.sub(r'http\S+', '', text)
  #removing the mention type from the data
  text = re.sub(r'http\S+', '', text)
  #tokenizing the words
  tokens=word_tokenize(text)
  #removing stopwords
  stop_words=set(stopwords.words('english'))
  filtered_token=[token for token in tokens if token not in stop_words and token.isalnum()]
#   #Lemmantize the token
#   lemmantizer=WordNetLemmatizer()
#   '''lemintizes words are looks like dogs---->dog ,happiness--->happy'''
#   lemmantized_words=[lemmantizer.lemmatize(token) for token in filtered_token]
  #join the token to make string
  preprocessed_text=''.join(filtered_token)
  return preprocessed_text


def convert_to_numeric(text):
  tokenizer = tf.keras.preprocessing.text.Tokenizer()
  tokenizer.fit_on_texts(data['text'])
  word_index = tokenizer.word_index
  sequences = tokenizer.texts_to_sequences(data['text'])
  padded_sequences = pad_sequences(sequences, maxlen=100)
  return padded_sequences

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('wordnet')



In [ ]:
nltk.data.path

In [ ]:
nltk.data.path.append('/usr/local/share/nltk_data')

tu=preprocess_text("@JetBlue is REALLY getting on my nerves !! 😡😡")
print(tu)

In [ ]:
convert_to_numeric(tu)

In [ ]:
df['text']=df['text'].apply(preprocess_text)

In [ ]:
df

# Using the Linear Regression method ♒

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer


#splitting the data
train_data,test_data,train_labels,test_labels=train_test_split(df['text'],df['airline_sentiment'],test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
train_data_counts = vectorizer.fit_transform(train_data)
test_data_counts = vectorizer.transform(test_data)



vec =TfidfTransformer()
train_vec=vec.fit_transform(train_data_counts)
test_vec=vec.transform(test_data_counts)

model=LogisticRegression(max_iter=10000)
model.fit(train_vec,train_labels)


In [ ]:
# Evaluate the model on the test set
accuracy = model.score(test_vec, test_labels)
print(f'Test accuracy: {accuracy:.4f}')

# Using Deep Learning ⚛


# Using Pretrained model BERT⚫

In [ ]:
#we need transformer 
!pip install transformers
#

In [ ]:
import torch
from transformers import TFBertForSequenceClassification,BertTokenizer,AdamW,get_linear_schedule_with_warmup,AutoModel,AutoTokenizer,BertModel


In [ ]:
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encoding=tokenizer(list(train_data['text']),truncation=True,padding=True)
valid_encoding=tokenizer(list(val_data['text']),truncation=True,padding=True)

sentiment_dict = {'positive': 0, 'negative': 1, 'neutral': 2}
train_labels = train_data['airline_sentiment'].map(sentiment_dict).values.astype('int64')
valid_labels = val_data['airline_sentiment'].map(sentiment_dict).values.astype('int64')

In [ ]:
print(len(train_labels))
print(len(valid_labels))
print(len(train_encoding))
print(len(valid_encoding))




In [ ]:
# Split the data into training and validation sets

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encoding), train_labels)).shuffle(len(train_labels)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_encoding), valid_labels)).batch(32)

# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Fine-tune the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)



# Balanced datasets : 

In [ ]:
df['airline_sentiment'].value_counts()

In [ ]:
import seaborn as sns
sns.countplot(data=df, x='airline_sentiment')


**It shows that the dataset is highly imbalanced by negative sentiment**

In [ ]:
df_new=df.drop(df[df.airline_sentiment == 'negative'].iloc[:6000].index)

In [ ]:
sns.countplot(data=df_new, x='airline_sentiment')


**After Dropping the 6000 data of negative sentiment the datasets seems to be balance**

In [ ]:
df_new['text']=df_new['text'].apply(preprocess_text)

In [ ]:
train_data, test_data = train_test_split(df_new, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encoding=tokenizer(list(train_data['text']),truncation=True,padding=True)
test_encoding=tokenizer(list(test_data['text']),truncation=True,padding=True)

sentiment_dict = {'positive': 0, 'negative': 1, 'neutral': 2}
train_labels = train_data['airline_sentiment'].map(sentiment_dict).values.astype('int64')
test_labels = test_data['airline_sentiment'].map(sentiment_dict).values.astype('int64')

In [ ]:
print(len(train_labels))
print(len(test_labels))
print(len(train_encoding))
print(len(test_encoding))

In [ ]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encoding), train_labels)).shuffle(len(train_labels)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_encoding), valid_labels)).batch(32)

# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Fine-tune the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

